In [ ]:
import tensorflow as tf
from sklearn import cross_validation
from sklearn import metrics as mt
from sklearn import utils as ut
import gc
import csv
import numpy as np
import os.path as path
from os import listdir
from PIL import Image

In [ ]:
# Data dir
imagesDir = './data/SampleFoodClassifier_Norm_100'

# Get list of files
files = [f for f in listdir(imagesDir) if path.isfile(path.join(imagesDir, f))]
predictions = [{'id': element} for element in files]

In [ ]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 100, 100, 3], name="x")
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
sdev= 0.01

# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# Layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.truncated_normal([5, 5, 3, 32], stddev=sdev), name="wc1"),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=sdev), name="wc2"),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.truncated_normal([25*25*64, 1024], stddev=sdev), name="wd1"),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.truncated_normal([1024, 2], stddev=sdev), name="outw")
}

biases = {
    'bc1': tf.Variable(tf.truncated_normal([32], stddev=sdev), name="bc1"),
    'bc2': tf.Variable(tf.truncated_normal([64], stddev=sdev), name="bc2"),
    'bd1': tf.Variable(tf.truncated_normal([1024], stddev=sdev), name="bd1"),
    'out': tf.Variable(tf.truncated_normal([2], stddev=sdev, name="outd"))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)
classes = tf.nn.softmax(pred)

init = tf.initialize_all_variables()

In [ ]:
# save the models
saveDir = './tensorflow/one_against_all'

In [ ]:
def loadData(files):
    return [np.array(Image.open(path.join(imagesDir, element))) for element in files]

In [ ]:
for currentLabel in range(9):
    saver = tf.train.Saver()
    
    print "Running predictions for label", currentLabel
    
    class_pred = []
    
    with tf.Session() as sess:
        saver.restore(sess, path.join(saveDir, "label-" + str(currentLabel) + ".ckpt"))
        
        for i in range(0, len(files), 50):
            if i+50 > len(files):
                data = loadData(files[i:])
            else:
                data = loadData(files[i:i+50])
            cls = sess.run(classes, feed_dict={x: data, keep_prob: 1.})
            class_pred.extend(cls)
    
    for i in range(len(files)):
        predictions[i][str(currentLabel)] = class_pred[i][1]

keys = predictions[0].keys()
with open(path.join('predictions','one_vs_all_class_prediction.csv'), 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(predictions)